# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: October 7th 2025

**Student Name**: arantxa angulo flores

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, count, window, current_timestamp
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming") \
    .master("spark://bb6818473482:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 00:59:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from arantxa.spark_utils import SparkUtils

# Use the generate_schema method to build the schema (instead of direct StructType)
log_schema = SparkUtils.generate_schema([
    ("timestamp", "string"),
    ("level", "string"),
    ("code", "string"), 
    ("message", "string")
])

print("Log schema defined using generate_schema method:\n")
log_schema

Log schema defined using generate_schema method:



StructType([StructField('timestamp', StringType(), True), StructField('level', StringType(), True), StructField('code', StringType(), True), StructField('message', StringType(), True)])

In [3]:
# Create streaming DataFrame from log files
log_stream_df = spark.readStream \
    .schema(log_schema) \
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .option("latestFirst", "false") \
    .load("/opt/spark/work-dir/data/structured_streaming_logs")  # Monitor this directory (maps to spark/data/structured_streaming_logs)

print("Streaming DataFrame created!")

Streaming DataFrame created!


In [4]:
# Display schema of streaming data
print("Streaming DataFrame Schema:")
log_stream_df.printSchema()

Streaming DataFrame Schema:
root
 |-- timestamp: string (nullable = true)
 |-- level: string (nullable = true)
 |-- code: string (nullable = true)
 |-- message: string (nullable = true)



In [5]:
# Display all logs in real-time
query = log_stream_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----+----+-------+
|timestamp          |level|code|message|
+-------------------+-----+----+-------+
|2025-10-10 03:31:57|INFO |200 |OK     |
+-------------------+-----+----+-------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-----+----+-------+
|timestamp          |level|code|message|
+-------------------+-----+----+-------+
|2025-10-10 03:32:00|INFO |200 |OK     |
+-------------------+-----+----+-------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+-----+----+---------+
|timestamp          |level|code|message  |
+-------------------+-----+----+---------+
|2025-10-10 03:32:03|ERROR|404 |Not Found|
+-------------------+-----+----+---------+

-------------------------------------------
Batch: 3
----------------------------

-------------------------------------------
Batch: 4
-------------------------------------------
+-------------------+-----+----+---------+
|timestamp          |level|code|message  |
+-------------------+-----+----+---------+
|2025-10-10 03:32:09|ERROR|404 |Not Found|
+-------------------+-----+----+---------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------------------+-----+----+-------+
|timestamp          |level|code|message|
+-------------------+-----+----+-------+
|2025-10-10 03:32:12|ERROR|200 |OK     |
+-------------------+-----+----+-------+

-------------------------------------------
Batch: 6
-------------------------------------------
+-------------------+-----+----+---------------------+
|timestamp          |level|code|message              |
+-------------------+-----+----+---------------------+
|2025-10-10 03:32:15|INFO |500 |Internal Server Error|
+-------------------+-----+----+---------------------+

-----------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
ERROR:py4j.clientserver:Exception occurred while shutting down connection
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self

-------------------------------------------
Batch: 79
-------------------------------------------
+-------------------+-----+----+---------+
|timestamp          |level|code|message  |
+-------------------+-----+----+---------+
|2025-10-10 03:35:54|ERROR|404 |Not Found|
+-------------------+-----+----+---------+

-------------------------------------------
Batch: 80
-------------------------------------------
+-------------------+-----+----+---------+
|timestamp          |level|code|message  |
+-------------------+-----+----+---------+
|2025-10-10 03:35:57|INFO |404 |Not Found|
+-------------------+-----+----+---------+

-------------------------------------------
Batch: 81
-------------------------------------------
+-------------------+-----+----+---------------------+
|timestamp          |level|code|message              |
+-------------------+-----+----+---------------------+
|2025-10-10 03:36:00|ERROR|500 |Internal Server Error|
+-------------------+-----+----+---------------------+

KeyboardInterrupt: 